In [ ]:
import numpy as np
import pandas as pd
from collections import OrderedDict, defaultdict
import glob
import sys 
sys.path.append('..')
sys.path.append('../../../../utils')
sys.path.append('../../../../third')
import gezi
from gezi import tqdm
tqdm.pandas()

In [ ]:
ACTIONS = ["read_comment", "like", "click_avatar",  "forward", "comment", "follow", "favorite"]
ACTIONS2 = ACTIONS + ['finish', 'stay']
HIS_ACTIONS = ACTIONS2 + [
    'pos', 'neg', 'unfinish', 'unstay'
]

In [ ]:
feeds = {}
def cache_feed():
  df = pd.read_csv('../input/feed_info.csv')
  df = df.fillna(-1)
  for row in tqdm(df.itertuples(), total=len(df), desc='feed_info'):
    row = row._asdict()
    feeds[row['feedid']] = row
cache_feed()

In [ ]:
with gezi.Timer('read train user_actions'):
  d = pd.read_feather('../input/user_action2.feather')
with gezi.Timer('read test and merge'):
  dt = pd.read_csv('../input/test_a.csv')
  try:
    dt1a = pd.read_csv('../input/v1/test_a.csv')
    dt1b = pd.read_csv('../input/v1/test_b.csv')
    dt = pd.concat([dt, dt1a, dt1b])
  except Exception:
    dtb = pd.read_csv('../input/test_b.csv')
    dt = pd.concat([dt, dtb])
dt['date_'] = 15
dshow = pd.concat([d[['userid', 'feedid', 'date_']],
                  dt[['userid', 'feedid', 'date_']]])
dshow = dshow.sort_values(['date_'], ascending=True)
d = d.sort_values(['date_'], ascending=True)

read train user_actions start
read train user_actions duration: 9.96309518814087
read test and merge start
read test and merge duration: 0.4151580333709717


In [ ]:
dshow[dshow.date_==15].head()

,userid,feedid,date_
139101,195031,53623,15
139098,132914,18257,15
139099,193946,103932,15
139095,22150,30475,15
139112,168537,66299,15


In [ ]:
# tag_info = {}
# for row in tqdm(fd.itertuples(), total=len(fd), desc='feed_info-manual_tag'):
#   row = row._asdict()
#   manual_tags = str(row['manual_tag_list']).split(';')
#   tag_info[int(row['feedid'])] = manual_tags

In [ ]:
d.head()

,userid,feedid,date_,device,read_comment,comment,like,play,stay,click_avatar,forward,follow,favorite
0,8,71474,1,1,0,0,1,500,5366,0,0,0,0
6031775,205838,66594,1,2,0,0,0,15390,15618,0,0,0,0
6031776,205838,54005,1,2,0,0,0,27607,27856,0,0,0,0
6031777,205838,88694,1,2,0,0,0,24583,25066,0,0,0,0
6031778,205838,29299,1,2,0,0,0,12897,13070,0,0,0,0


In [ ]:
df = d

In [ ]:
def write_corpus(day):
  d = df[df.date_ < day]
  dfs = {}
  t = tqdm(ACTIONS)
  for action in t:
    t.set_postfix({'action': action})
    dfs[action] = d[d[action] == 1].groupby(['userid'])['feedid'].progress_apply(list).reset_index(name=action)
  dfs['finish'] = d[d.finish_rate > 0.99].groupby(['userid'])['feedid'].progress_apply(list).reset_index(name='finish')
  dfs['unfinish'] = d[d.finish_rate < 0.01].groupby(['userid'])['feedid'].progress_apply(list).reset_index(name='unfinish')
  dfs['stay'] = d[d.stay_rate > 1].groupby(['userid'])['feedid'].progress_apply(list).reset_index(name='stay')
  dfs['unstay'] = d[d.stay_rate < 0.01].groupby(['userid'])['feedid'].progress_apply(list).reset_index(name='unstay')
  dfs['neg'] = d[d.actions == 0].groupby(['userid'])['feedid'].progress_apply(list).reset_index(name='neg')
  dfs['pos'] = d[d.actions > 0].groupby(['userid'])['feedid'].progress_apply(list).reset_index(name='pos')
  # dfs['show'] = dshow.groupby(['userid'])['feedid'].progress_apply(list).reset_index(name='show')
  for action in tqdm(dfs):
    with open(f'../input/{day}/doc_{action}.txt', 'w') as f:
      for l in dfs[action][action].values:
        print(' '.join(map(str, l)), file=f)
    keys = ['author', 'singer', 'song']
    keys_ = ['authorid', 'bgm_singer_id', 'bgm_song_id']
    for key, key_ in zip(keys, keys_):
      with open(f'../input/{day}/{key}_{action}.txt', 'w') as f:
        for l in dfs[action][action].values:
          print(' '.join(map(lambda x:str(feeds[x][key_]), l)), file=f)
  return dfs


In [ ]:
dfs_14 = write_corpus(14)

  0%|          | 0/7 [00:00<?, ?it/s]

In [ ]:
dfs_15 = write_corpus(15)

In [ ]:
# dfs['read_comment'].merge(dfs['comment'], on='userid')

,userid,feedids_read_comment,feedids_comment
0,756,"[89722, 90203, 81546, 54316, 37375, 1694, 9690...",[40057]
1,970,"[6169, 68872, 6090, 86800, 70661]",[68872]
2,1415,"[49092, 64373, 28462]","[64373, 28462]"
3,1737,[55526],[55526]
4,1947,"[8147, 14833, 86324, 50713, 259, 47956]",[14833]
...,...,...,...
658,249345,"[106501, 27533, 13208, 54723, 72380, 24426, 52...","[106501, 72380, 27761, 25487, 92822, 86993]"
659,249461,"[91748, 95743, 20928, 13539, 38788, 9923, 2107...",[20928]
660,249662,[68955],[68955]
661,249990,"[82869, 100968, 32916]",[100968]
